In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  |     Proc size: 119.0 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


In [ ]:
cd /content/drive/My Drive/Colab_Notebooks/BCI/p3net/bauka

/content/drive/My Drive/Colab_Notebooks/BCI/p3net/bauka


In [ ]:
!ls

dataloader_bauka.ipynb	models_bauka.ipynb    train_utils_bauka.ipynb
dataloader_bauka.py	models_bauka.py       train_utils_bauka.py
data_NUsubjects.pickle	__pycache__
model_bauyrzhan		train_nn_bauka.ipynb


In [ ]:
!cat 'models_bauka.py'
!cat 'dataloader_bauka.py'
!cat 'train_utils_bauka.py'

import sys
sys.path.append('/content/drive/My\ Drive/Colab_Notebooks/BCI/p3net/bauka')

# -*- coding: utf-8 -*-
"""models_bauka.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Vdb4SJpmfHOnfHnsK_DbQR8fvlvLR6hQ
"""

import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
import pdb

# %%
def compute_conv_dim(dim_size, kernel_size, padding, stride):
    # compute the output dimensions of a convolutional layer
    out_dim = int((dim_size - kernel_size + 2 * padding) / stride + 1)
    return out_dim

# %%
class CNN2D(torch.nn.Module):

    def __init__(self,
                 input_size,  # (1, 16, 76),
                 kernel_size=[3, 3],
                 conv_channels=[1, 8],
                 dense_size=256,
                 dropout=0.1):

        super(CNN2D, self).__init__()
        self.cconv = []
        self.MaxPool = nn.MaxPool2d((1, 2), (1, 2))
        self.ReLU = nn.ReLU()
        self.Dropout = nn.Dropout(dropout)
        self.batchnorm = []

        for jj in 

In [ ]:
import torch 
import pandas as pd 
import pickle

from dataloader_bauka import getTorch, EEGDataLoader
from train_utils_bauka import train_model
from models_bauka import *

In [ ]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    print("CUDA available!")
    dev = torch.device("cuda")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')


get_data = getTorch.get_data 

# Load ERP data
dname = dict(nu   = 'data_NUsubjects.pickle')

#%% Hyperparameter settings 

num_epochs = 30
batch_size = 8
verbose = 2
learning_rate = 1e-2
weight_decay = 1e-4                   

CUDA available!


In [ ]:
print(dname)

{'nu': 'data_NUsubjects.pickle'}


In [ ]:
!pip3 install mne

In [ ]:
for itemname, filename in dname.items():

    print('working with', filename)
    iname = itemname + '__'   
    
    if filename =='data_NUsubjects.pickle':
        d  =  {'conv_channels': [1, 256, 128, 64, 32, 16, 8], # convolutional layers and number of filters in each layer 
               'kernel_size':   [3, 3, 3, 3, 3, 3], # kernel size 
               'num_layers':    1, # number of LSTM layers
               'hidden_size':   128} # number of neurons in each LSTM layer 
  
    
    # EEG data loader 
    dd = EEGDataLoader(filename)
    
    # load subject specific data with subject data indicies  
    s = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
    data = dd.subject_specific(s)
    print(data[0].keys())
        
    #% identify input size (channel x timepoints)
    timelength = data[0]['xtrain'].shape[2]
    chans      = data[0]['xtrain'].shape[1]
 
    datum = {}
    # get torch data loaders for each subject data 
    for ii in range(len(data)):
      datum[ii] = get_data(data[ii], batch_size, image = True, lstm = False, raw = False)
   
    # used for storing results later 
    results, models = {}, {}  
    table = pd.DataFrame(columns = ['Train_Loss', 'Val_Loss', 'Train_Acc', 
                                        'Val_Acc', 'Test_Acc', 'Epoch'])
    
    # for each subject in a given data type perform model selection
    for subjectIndex in datum:    
        dset_loaders = datum[subjectIndex]['dset_loaders']
        dset_sizes   = datum[subjectIndex]['dset_sizes']      
        
        input_size = datum[subjectIndex]['test_data']['x_test'].shape

        # define the encoder model                  
        encoder = CNN2DEncoder(kernel_size = d['kernel_size'], 
                               conv_channels = d['conv_channels'])      
        # we need to pass a sample data to get the output dimensionality of CNN 
        with torch.no_grad():
            x = torch.randn(input_size)
            outdim = encoder(x)
            batch_size, chans, H, W = outdim.size()
            
        # define the encoder-decoder model 
        model = CNNLSTM(input_size = H*W, 
                        cnn = encoder,
                        hidden_size = d['hidden_size'], 
                        num_layers  = d['num_layers'], 
                        batch_size  = 8,
                        dropout     = 0.2)      
        
        # define the optimizer and the loss function 
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
        criterion = torch.nn.CrossEntropyLoss()
        
        # move to GPU/CPU
        model.to(dev)  
        criterion.to(dev)           
        
        # the training loop
        best_model, train_losses, val_losses, train_accs, val_accs, info = train_model(model, dset_loaders, 
                                                                                       dset_sizes, criterion, 
                                                                                       optimizer, dev, 
                                                                                       lr_scheduler = None, 
                                                                                       num_epochs = num_epochs, 
                                                                                       verbose = verbose)    
         
        # here train_model returns the best_model which is saved for a later use below        
        # we could immediately evaluate the best model on the test as
        x_test = datum[subjectIndex]['test_data']['x_test'] 
        y_test = datum[subjectIndex]['test_data']['y_test'] 
        
        h=best_model.init_hidden(x_test.shape[0])
        preds = best_model(x_test.to(dev),h)    
        preds_class = preds.data.max(1)[1]
        
        # accuracy metric 
        corrects = torch.sum(preds_class == y_test.data.to(dev))     
        test_acc = corrects.cpu().numpy()/x_test.shape[0]    
        print("Test Accuracy :", test_acc)    
        
        # save results       
        tab = dict(Train_Loss = train_losses[info['best_epoch']], 
                   Val_Loss   = val_losses[info['best_epoch']],
                   Train_Acc  = train_accs[info['best_epoch']],
                   Val_Acc    = val_accs[info['best_epoch']],   
                   Test_Acc   = test_acc, 
                   Epoch      = info['best_epoch'] + 1) 
        
        subjectIndexx = subjectIndex + 1
        table.loc[subjectIndexx] = tab
        
        results[subjectIndexx] = dict(train_losses = train_losses, val_losses = val_losses,
                                    train_accs = train_accs,     val_accs =  val_accs,                                
                                    ytrain = info['ytrain'],     yval= info['yval'])      
          
        # save models   
        fname = 'model_bauyrzhan' + iname + 'S'+ str(subjectIndexx) + '_CNNLSTM_model_'
        torch.save(best_model.state_dict(), fname) 
         
        print('::: saving subject {} ::: \n {}'.format(subjectIndexx, table))         
        result_lstm_subspe = dict(table = table, results = results)          
          
    fname = 'model_bauyrzhan' + iname + '__S'+str(subjectIndexx)  + '_CNNLSTM_subspe_results'
    
    with open(fname, 'wb') as fp:
        pickle.dump(result_lstm_subspe, fp)  

working with data_NUsubjects.pickle
Loading subjects: 0
Loading subjects: 1
Loading subjects: 2
Loading subjects: 3
Loading subjects: 4
Loading subjects: 5
Loading subjects: 6
Loading subjects: 7
Loading subjects: 8
Loading subjects: 9
Loading subjects: 10
dict_keys(['xtrain', 'xvalid', 'xtest', 'ytrain', 'yvalid', 'ytest'])
Input data shape torch.Size([4732, 1, 16, 76])
Input data shape torch.Size([4629, 1, 16, 76])
Input data shape torch.Size([2435, 1, 16, 76])
Input data shape torch.Size([7727, 1, 16, 76])
Input data shape torch.Size([4026, 1, 16, 76])
Input data shape torch.Size([4374, 1, 16, 76])
Input data shape torch.Size([1302, 1, 16, 76])
Input data shape torch.Size([1393, 1, 16, 76])
Input data shape torch.Size([4910, 1, 16, 76])
Input data shape torch.Size([4449, 1, 16, 76])
Input data shape torch.Size([4458, 1, 16, 76])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 1/30
train loss: 0.0870, acc: 0.5325
val loss: 0.0868, acc: 0.5376
Epoch 2/30
train loss: 0.0862, acc: 0.5306
val loss: 0.0869, acc: 0.4734
Epoch 3/30
train loss: 0.0864, acc: 0.5473
val loss: 0.0855, acc: 0.5680
Epoch 4/30
train loss: 0.0866, acc: 0.5338
val loss: 0.0861, acc: 0.5385
Epoch 5/30
train loss: 0.0865, acc: 0.5374
val loss: 0.0867, acc: 0.5385
Epoch 6/30
train loss: 0.0868, acc: 0.5256
val loss: 0.0870, acc: 0.5089
Epoch 7/30
train loss: 0.0869, acc: 0.5311
val loss: 0.0902, acc: 0.4607
Epoch 8/30
train loss: 0.0868, acc: 0.5281
val loss: 0.0863, acc: 0.5385
Epoch 9/30
train loss: 0.0867, acc: 0.5357
val loss: 0.0865, acc: 0.5385
Epoch 10/30
train loss: 0.0869, acc: 0.5237
val loss: 0.0866, acc: 0.5376
Epoch 11/30
train loss: 0.0869, acc: 0.5370
val loss: 0.0863, acc: 0.5385
Epoch 12/30
train loss: 0.0867, acc: 0.5313
val loss: 0.0866, acc: 0.5385
Epoch 13/30
train loss: 0.0868, acc: 0.5254
val loss: 0.0864, acc: 0.5385
Epoch 14/30
train loss: 0.0868, acc: 0.5351
val

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


train loss: 0.0826, acc: 0.6103
val loss: 0.0926, acc: 0.5866
Epoch 2/30
train loss: 0.0773, acc: 0.6658
val loss: 0.0876, acc: 0.6253
Epoch 3/30
train loss: 0.0736, acc: 0.6874
val loss: 0.0889, acc: 0.6382
Epoch 4/30
train loss: 0.0728, acc: 0.6922
val loss: 0.0759, acc: 0.6667
Epoch 5/30
train loss: 0.0713, acc: 0.7027
val loss: 0.0786, acc: 0.6374
Epoch 6/30
train loss: 0.0700, acc: 0.7081
val loss: 0.0848, acc: 0.5978
Epoch 7/30
train loss: 0.0670, acc: 0.7313
val loss: 0.0806, acc: 0.6374
Epoch 8/30
train loss: 0.0639, acc: 0.7552
val loss: 0.0822, acc: 0.6615
Epoch 9/30
train loss: 0.0626, acc: 0.7613
val loss: 0.0789, acc: 0.6581
Epoch 10/30
train loss: 0.0578, acc: 0.7853
val loss: 0.0794, acc: 0.6667
Epoch 11/30
train loss: 0.0559, acc: 0.7930
val loss: 0.0844, acc: 0.6675
Epoch 12/30
train loss: 0.0527, acc: 0.8105
val loss: 0.1052, acc: 0.6563
Epoch 13/30
train loss: 0.0523, acc: 0.8190
val loss: 0.0850, acc: 0.6460
Epoch 14/30
train loss: 0.0487, acc: 0.8315
val loss: 0.09

...

train loss: 0.0841, acc: 0.5829
val loss: 0.0812, acc: 0.6553
Epoch 2/30
train loss: 0.0797, acc: 0.6448
val loss: 0.0819, acc: 0.6464
Epoch 3/30
train loss: 0.0787, acc: 0.6536
val loss: 0.0843, acc: 0.6050
Epoch 4/30
train loss: 0.0766, acc: 0.6587
val loss: 0.0834, acc: 0.6156
Epoch 5/30
train loss: 0.0768, acc: 0.6688
val loss: 0.0766, acc: 0.6796
Epoch 6/30
train loss: 0.0739, acc: 0.6872
val loss: 0.0826, acc: 0.6164
Epoch 7/30
train loss: 0.0737, acc: 0.6896
val loss: 0.0798, acc: 0.6642
Epoch 8/30
train loss: 0.0729, acc: 0.6988
val loss: 0.0829, acc: 0.6164
Epoch 9/30
train loss: 0.0711, acc: 0.7116
val loss: 0.0774, acc: 0.6837
Epoch 10/30
train loss: 0.0685, acc: 0.7218
val loss: 0.0867, acc: 0.6642
Epoch 11/30
train loss: 0.0678, acc: 0.7310
val loss: 0.0770, acc: 0.6821
Epoch 12/30
train loss: 0.0655, acc: 0.7470
val loss: 0.0787, acc: 0.6869
Epoch 13/30
train loss: 0.0631, acc: 0.7633
val loss: 0.0872, acc: 0.6764
Epoch 14/30
train loss: 0.0612, acc: 0.7656
val loss: 0.08

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch 1/30
train loss: 0.0877, acc: 0.5118
val loss: 0.0872, acc: 0.4987
Epoch 2/30
train loss: 0.0855, acc: 0.5651
val loss: 0.0870, acc: 0.5193
Epoch 3/30
train loss: 0.0850, acc: 0.5759
val loss: 0.0868, acc: 0.5382
Epoch 4/30
train loss: 0.0843, acc: 0.5817
val loss: 0.0882, acc: 0.5310
Epoch 5/30
train loss: 0.0837, acc: 0.5898
val loss: 0.0861, acc: 0.5409
Epoch 6/30
train loss: 0.0843, acc: 0.5729
val loss: 0.0874, acc: 0.5049
Epoch 7/30
train loss: 0.0860, acc: 0.5547
val loss: 0.0883, acc: 0.4969
Epoch 8/30
train loss: 0.0861, acc: 0.5516
val loss: 0.0872, acc: 0.5220
Epoch 9/30
train loss: 0.0856, acc: 0.5642
val loss: 0.0888, acc: 0.5238
Epoch 10/30
train loss: 0.0850, acc: 0.5774
val loss: 0.0868, acc: 0.5732
Epoch 11/30
train loss: 0.0865, acc: 0.5253
val loss: 0.0877, acc: 0.5049
Epoch 12/30
train loss: 0.0869, acc: 0.5345
val loss: 0.0861, acc: 0.5067
Epoch 13/30
train loss: 0.0851, acc: 0.5720
val loss: 0.0869, acc: 0.5400
Epoch 14/30
train loss: 0.0850, acc: 0.5797
val

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


train loss: 0.0867, acc: 0.5334
val loss: 0.0876, acc: 0.5004
Epoch 2/30
train loss: 0.0867, acc: 0.5354
val loss: 0.0868, acc: 0.5085
Epoch 3/30
train loss: 0.0873, acc: 0.5139
val loss: 0.0870, acc: 0.4996
Epoch 4/30
train loss: 0.0868, acc: 0.5144
val loss: 0.0871, acc: 0.5094
Epoch 5/30
train loss: 0.0868, acc: 0.5287
val loss: 0.0856, acc: 0.5552
Epoch 6/30
train loss: 0.0860, acc: 0.5529
val loss: 0.0887, acc: 0.5076
Epoch 7/30
train loss: 0.0865, acc: 0.5431
val loss: 0.0870, acc: 0.5076
Epoch 8/30
train loss: 0.0865, acc: 0.5328
val loss: 0.0872, acc: 0.5274
Epoch 9/30
train loss: 0.0866, acc: 0.5377
val loss: 0.0871, acc: 0.5327
Epoch 10/30
train loss: 0.0867, acc: 0.5379
val loss: 0.0871, acc: 0.5076
Epoch 11/30
train loss: 0.0869, acc: 0.5105
val loss: 0.0870, acc: 0.5229
Epoch 12/30
train loss: 0.0870, acc: 0.5070
val loss: 0.0873, acc: 0.5085
Epoch 13/30
train loss: 0.0871, acc: 0.5087
val loss: 0.0869, acc: 0.5202
Epoch 14/30
train loss: 0.0872, acc: 0.5092
val loss: 0.08